Construindo um grafo automaticamente
====================================



## Introdução



Neste notebook nós vamos dar o primeiro passo para construir nossa rede neural artificial. Neste primeiro passo, nós vamos criar uma classe que gera automaticamente o nosso `grafo computacional`. O grafo computacional é o grafo que representa todas as operações matemáticas que ocorreram ao se computar um certo valor $y$. O grafo computacional é um passo necessário pois será baseado nele que iremos computar os gradientes locais necessários para realizar o `backpropagation`.



In [1]:
try:
    import graphviz
except ModuleNotFoundError:
    import sys
    !{sys.executable} -m pip install graphviz

## Importações



In [2]:
from funcoes import plota_grafo

## Código e discussão



### Primeiros passos



A base de tudo será uma classe chamada `Valor`. Vamos começar pelo básico!



In [3]:
class Valor:
    def __init__(self, data):
        self.data = data

    def __repr__(self):
        return f"Valor(data={self.data})"

Vamos testar nossa classe!



In [4]:
x1 = Valor(60)
print(x1)
print(x1.data)

Valor(data=60)
60


### Os dunders `__add__` e `__mul__`



Observe que não conseguimos adicionar nem multiplicar objetos criados com a classe `Valor`.



In [5]:
a = Valor(10)
b = Valor(5)

In [7]:
print(a + b)

TypeError: unsupported operand type(s) for +: 'Valor' and 'Valor'

In [8]:
print(a * b)

TypeError: unsupported operand type(s) for *: 'Valor' and 'Valor'

Ué&#x2026; porque não conseguimos? Não conseguimos pois o Python (ainda) não é vidente. Ele lá vai saber como adicionar ou multiplicar algo que você criou? Pra você parece óbvio que valores podem ser adicionados ou multiplicados, mas para o Python ele nem sabe o que significa a palavra `Valor`&#x2026;

Como sempre, temos que contar para o programa o que queremos que aconteça quando usarmos os operadores `+` e `*`. Quem faz isso são os dunders `__add__` e `__mul__`.



In [9]:
class Valor:
    def __init__(self, data):
        self.data = data

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        saida = Valor(self.data + outro_valor.data)
        return saida

    def __mul__(self, outro_valor):
        saida = Valor(self.data * outro_valor.data)
        return saida

Vamos testar!



In [10]:
a = Valor(10)
b = Valor(5)

print(a + b)
print(a * b)

Valor(data=15)
Valor(data=50)


### Registrando os progenitores



Nosso objetivo é construir um grafo computacional. Em um grafo computacional, um certo vértice pode ter um ou mais vértices progenitores (são seus pais/mães). Nós não podemos perder essa informação quando formos construir um grafo, então precisamos incluir essa informação na nossa classe.



In [11]:
class Valor:
    def __init__(self, data, progenitor=()):
        self.data = data
        self.progenitor = progenitor

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        data = self.data + outro_valor.data
        progenitor = (self, outro_valor)
        saida = Valor(data, progenitor)
        return saida

    def __mul__(self, outro_valor):
        data = self.data * outro_valor.data
        progenitor = (self, outro_valor)
        saida = Valor(data, progenitor)
        return saida

Vamos testar!



In [12]:
a = Valor(10)
b = Valor(5)

print(a.data)
print(a.progenitor)

c = a + b
d = c * a

print(c)
print(a)
print(d.progenitor)

10
()
Valor(data=15)
Valor(data=10)
(Valor(data=15), Valor(data=10))


### Registrando o operador mãe



Em um grafo computacional, um vértice pode ter um operador mãe. O operador mãe é o operador que foi usado para gerar o vértice.



In [13]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        data = self.data + outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "+"
        saida = Valor(data, progenitor, operador_mae)
        return saida

    def __mul__(self, outro_valor):
        data = self.data * outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "*"
        saida = Valor(data, progenitor, operador_mae)
        return saida

Vamos testar!



In [14]:
a = Valor(10)
b = Valor(5)

c = a + b
d = c * a

print(c)
print(c.progenitor)
print(c.operador_mae)

Valor(data=15)
(Valor(data=10), Valor(data=5))
+


### Plotando o primeiro grafo



Vamos plotar nosso primeiro grafo!



In [15]:
a = Valor(2)
b = Valor(-3)
c = Valor(10)

d = a * b
e = d + c

grafo = plota_grafo(e)
print(grafo)

digraph {
	graph [rankdir=LR]
	1386056275472 [label="{ data 2.0000 }" shape=record]
	1386056276096 [label="{ data -6.0000 }" shape=record]
	"1386056276096*" [label="*"]
	"1386056276096*" -> 1386056276096
	1386056277152 [label="{ data -3.0000 }" shape=record]
	1386056274752 [label="{ data 10.0000 }" shape=record]
	1386056274848 [label="{ data 4.0000 }" shape=record]
	"1386056274848+" [label="+"]
	"1386056274848+" -> 1386056274848
	1386056275472 -> "1386056276096*"
	1386056274752 -> "1386056274848+"
	1386056276096 -> "1386056274848+"
	1386056277152 -> "1386056276096*"
}



### Registrando o rótulo



Nosso grafo seria mais legível se tivéssemos rótulos indicando o que é cada vértice. Vamos incluir essa informação na nossa classe.



In [16]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        data = self.data + outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "+"
        saida = Valor(data, progenitor, operador_mae)
        return saida

    def __mul__(self, outro_valor):
        data = self.data * outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "*"
        saida = Valor(data, progenitor, operador_mae)
        return saida

Vamos testar!



In [17]:
a = Valor(2, rotulo="a")
b = Valor(-3, rotulo="b")
c = Valor(10, rotulo="c")

d = a * b
e = d + c

d.rotulo = "d"
e.rotulo = "e"

grafo = plota_grafo(e)
print(grafo)

digraph {
	graph [rankdir=LR]
	1386055580160 [label="{ a | data 2.0000 }" shape=record]
	1386055578432 [label="{ c | data 10.0000 }" shape=record]
	1386055578960 [label="{ b | data -3.0000 }" shape=record]
	1386055580592 [label="{ d | data -6.0000 }" shape=record]
	"1386055580592*" [label="*"]
	"1386055580592*" -> 1386055580592
	1386055580112 [label="{ e | data 4.0000 }" shape=record]
	"1386055580112+" [label="+"]
	"1386055580112+" -> 1386055580112
	1386055578432 -> "1386055580112+"
	1386055580160 -> "1386055580592*"
	1386055578960 -> "1386055580592*"
	1386055580592 -> "1386055580112+"
}



### Refazendo o grafo que fizemos na aula anterior



Na aula anterior nós fizemos um grafo computacional para aprender como funciona o backpropagation. Vamos refazer ele aqui!

In [18]:
x1= Valor(60, rotulo= "x1")
x2= Valor(24, rotulo= "x2")
w1= Valor(10, rotulo= "w1")
w2= Valor(5, rotulo= "w2")
w3= Valor(2, rotulo= "w3")
b= Valor(7, rotulo= "b")

s1= x1*w1
s1.rotulo= "s1"

s2= x2*w2
s2.rotulo= "s2"

n= s1+s2
n.rotulo= "n"

k= n+b
k.rotulo= "k"

y= k* w3
y.rotulo="y"

grafo= plota_grafo(y)
print(grafo)

digraph {
	graph [rankdir=LR]
	1386056277536 [label="{ k | data 727.0000 }" shape=record]
	"1386056277536+" [label="+"]
	"1386056277536+" -> 1386056277536
	1386056275040 [label="{ w2 | data 5.0000 }" shape=record]
	1386056277104 [label="{ n | data 720.0000 }" shape=record]
	"1386056277104+" [label="+"]
	"1386056277104+" -> 1386056277104
	1386056275088 [label="{ s1 | data 600.0000 }" shape=record]
	"1386056275088*" [label="*"]
	"1386056275088*" -> 1386056275088
	1386055544496 [label="{ y | data 1454.0000 }" shape=record]
	"1386055544496*" [label="*"]
	"1386055544496*" -> 1386055544496
	1386056276720 [label="{ s2 | data 120.0000 }" shape=record]
	"1386056276720*" [label="*"]
	"1386056276720*" -> 1386056276720
	1386056275232 [label="{ x2 | data 24.0000 }" shape=record]
	1386056276912 [label="{ w3 | data 2.0000 }" shape=record]
	1386056277392 [label="{ b | data 7.0000 }" shape=record]
	1386055510960 [label="{ w1 | data 10.0000 }" shape=record]
	1386056275952 [label="{ x1 | data 60.0000 }" 

## Conclusão



_Bom, antes de concluir algo, devo informar que não instalei (com o Eric) o que era necessário para formar um grafo. Entretanto, consegui visualizar pelos computadores alheios._

Focando no último grafo, podemos ver que o código acima cria uma rede neural artificial com duas camadas e nós de saída. A primeira camada consiste em dois nós (s1 e s2), cada um com um valor de entrada (x1 e x2) e um peso associado (w1 e w2). Os valores resultantes são adicionados a um nó (n), adicionado ao termo de polarização (b) da camada de saída. O nó de saída (y) é obtido multiplicando o nó de saída (k) da camada anterior pelo peso (w3).  

O grafo resultante representa uma árvore de expressão de uma rede neural, onde os nós representam valores numéricos e as conexões representam as operações matemáticas (multiplicação e adição) que os conectam. Tendo isso em vista, o identificador de cada nó é utilizado para identificar sua função na rede. 

Por fim, quando pensamos em `backpropagation`, é preciso ter em mente que esse algoritmo de aprendizado é usado para ajustar os pesos da rede neural durante o treinamento. Ele funciona alimentando o erro calculado na camada de saída da rede de volta para as camadas anteriores, ajustando os pesos conforme o tamanho do erro e a sensibilidade de cada nó aos valores de entrada e seus pesos associados. Isso será importante futuramente, porque o algoritmo passa por várias iterações até que o erro seja minimizado e a rede neural possa produzir as saídas desejadas para a entrada de treinamento.